In [14]:
!pip install spotipy


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
# import libraries
import numpy as np
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from tqdm import tqdm  
import time

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

### Spotify API

In [ ]:
# Set up API credentials
client_id = ""
client_secret = ""

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

### Get Tracks

In [ ]:
# Get a list of n tracks each year from start to end year (inclusive)
def top_tracks_by_year(start, end, top_n):
    results_by_year = {}

    for year in range(start, end+1):
        query = f'year:{year}'
        track_info = sp.search(q=query, type='track', limit=top_n)
        top_tracks = []

        for t in track_info['tracks']['items']:
            track_name = t['name']
            artists = t['artists'][0]['name']
            popularity = t['popularity']
            track_id = t['id']
            
            top_tracks.append({
                'name': track_name,
                'artists': artists,
                'popularity': popularity,
                'id': track_id
            })

        results_by_year[year] = top_tracks

    return results_by_year

# Change year and top n tracks here
top_tracks = top_tracks_by_year(1950, 2020, 20)

### Convert to DF

In [ ]:
data = []

# Append tracks and their features to data
for year in top_tracks:
    tracks = top_tracks[year]
    rank = 1

    for track in tracks:
        data.append({
            'Year': year,
            'Rank': rank,
            'Track Name': track['name'],
            'Artist(s)': track['artists'],
            'Popularity': track['popularity'],
            'Track ID': track['id']
        })
        rank += 1

# Create the DataFrame
df = pd.DataFrame(data)

# Create csv file, move this down after adding genre to df
### Delete this file each time you add new things to df, so it updates the file
df.to_csv('spotify_top_tracks_by_year.csv', index=False)  

### Create Genres

In [17]:
# Create a list to store genres
genres_list = []

# Loop through the DataFrame to get genres from artist info
for idx, row in tqdm(df.iterrows(), total=len(df)):
    try:
        artist_name = row['Artist(s)']
        search_artist = sp.search(q=f"artist:{artist_name}", type='artist')
        artist_items = search_artist['artists']['items']
        
        if artist_items:
            artist_info = artist_items[0]
            genres = artist_info['genres']
            genres_list.append(genres[0] if genres else None)  # Pick the first genre if exists
        else:
            genres_list.append(None)
            
    except Exception as e:
        genres_list.append(None)

    time.sleep(0.05)  # Prevent hitting rate limit

# Add genre column to your DataFrame
df['Genre'] = genres_list

# Save the updated DataFrame
df.to_csv('spotify_top_tracks_with_genres.csv', index=False)

NameError: name 'df' is not defined